<a href="https://colab.research.google.com/github/Adil1979/P6_Tag_Recommendation_Stackoverflow/blob/master/04_APP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Projet6: Catégorisation automatique de questions**

## **1.Librairies**

In [124]:
!pip install gensim # Gensim is an open-source library for unsupervised topic modeling and natural language processing

In [125]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.lines import Line2D
import matplotlib as mpl
#import chart_studio.plotly.plotly as py
from collections import Counter
#import plotly.plotly as py
import seaborn as sns
import datetime as dt
import calendar as cld
from scipy import stats
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import scale
from os import listdir
import glob
import missingno as msno
#from feature_engine.categorical_encoders import OneHotCategoricalEncoder
from sklearn.decomposition import PCA
from matplotlib import dates
import nltk
import re
from nltk.corpus import stopwords 
nltk.download('stopwords')
from nltk.stem.porter import PorterStemmer 
import string
from collections import Counter
from bs4 import BeautifulSoup
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from ast import literal_eval 
from sklearn.externals import joblib
from joblib import load, dump
stop_words = set(stopwords.words('english'))
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.stem.snowball import EnglishStemmer
lemma = WordNetLemmatizer().lemmatize
ps = PorterStemmer()
stemmer = EnglishStemmer()
from joblib import load
random_state = 747

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [126]:
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Package cess_esp is already up-to-date!
[nltk_data]    | Downloading packag

True

In [127]:
set_alpha = set(['a', 'b', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '-'])

## **2.Data**

In [128]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [129]:
model_lr = joblib.load('/content/drive/My Drive/CentraleSupelec_OCR/P6/reg_log_saved.joblib')

In [130]:
dict_tags_500 = joblib.load('/content/drive/My Drive/CentraleSupelec_OCR/P6/list_tag_key_500.joblib')
dict_tags_500

{'.htaccess',
 '.net',
 '.net-3.5',
 '.net-core',
 'actionscript-3',
 'activerecord',
 'ajax',
 'algorithm',
 'amazon-ec2',
 'amazon-s3',
 'amazon-web-services',
 'android',
 'android-activity',
 'android-emulator',
 'android-fragments',
 'android-gradle-plugin',
 'android-intent',
 'android-layout',
 'android-studio',
 'angular',
 'angular-material',
 'angular5',
 'angularjs',
 'animation',
 'ansible',
 'ant',
 'apache',
 'apache-kafka',
 'apache-spark',
 'apache-spark-sql',
 'api',
 'architecture',
 'arraylist',
 'arrays',
 'asp.net',
 'asp.net-core',
 'asp.net-core-2.0',
 'asp.net-core-mvc',
 'asp.net-mvc',
 'asp.net-mvc-2',
 'asp.net-mvc-3',
 'asp.net-mvc-4',
 'asp.net-web-api',
 'assembly',
 'async-await',
 'asynchronous',
 'audio',
 'authentication',
 'autocomplete',
 'awk',
 'aws-lambda',
 'azure',
 'bash',
 'batch-file',
 'binary',
 'binding',
 'boost',
 'bootstrap-4',
 'browser',
 'build',
 'button',
 'c',
 'c#',
 'c++',
 'c++11',
 'c++14',
 'c++17',
 'caching',
 'cakephp',
 '

In [131]:
type(dict_tags_500)

set

In [132]:
st_tags_500  = " ".join(dict_tags_500)
st_tags_500

'visual-studio-2010 safari asp.net-mvc-3 iis shell static redirect .net-core utf-8 entity-framework cocoa xml binding maven react-router haskell jackson string oauth opengl csv keras erlang mysql mobile sbt configuration fonts asp.net-core-2.0 apache-kafka nhibernate listview facebook sorting unix error-handling merge oop ajax regex qt html django github time firebase ruby-on-rails-4 environment-variables scikit-learn amazon-web-services ios web-services ipad jdbc file-io timer intellij-idea julia audio jaxb websocket https date macros authentication syntax ios8 exception iterator graph email xcode user-interface multithreading sql-server hadoop jsf jakarta-ee image ruby canvas c++11 eclipse-plugin floating-point delphi group-by google-chrome iis-7 rxjs dockerfile sed colors optimization heroku for-loop ubuntu zend-framework java logging laravel pandas vue.js windows google-maps mvvm lisp model eclipse f# migration parallel-processing java-stream typescript compilation ssis dictionary 

In [133]:
type(st_tags_500)

str

In [134]:
question_test = 'C# gRPC client - name resolution failure <p>A client is running our C# gRPC client on a corporate network, behind an HTTP proxy. The <code>http_proxy</code> environment variable is configured, but nevertheless he sees an error message <code>Name resolution failure</code> when attempting to connect to the server on the internet. </p>\n\n<p>DNS resolution from the same machine works fine using nslookup. </p>\n\n<p>Any ideas what I can do to investigate this problem?</p>\n'
question_test

'C# gRPC client - name resolution failure <p>A client is running our C# gRPC client on a corporate network, behind an HTTP proxy. The <code>http_proxy</code> environment variable is configured, but nevertheless he sees an error message <code>Name resolution failure</code> when attempting to connect to the server on the internet. </p>\n\n<p>DNS resolution from the same machine works fine using nslookup. </p>\n\n<p>Any ideas what I can do to investigate this problem?</p>\n'

In [135]:
type(question_test)

str

In [136]:
def preprocessing(text):
  #HTML characters removing
  def remove_html(text):
    html_regex = re.compile('<.*?>') #Compile regular expresions
    return re.sub(html_regex, ' ', str(text)) # Replace regex by ' '
  text0 = remove_html(text)
  

  #URL removing
  def remove_url(text0):
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    return re.sub(url_regex, ' ', str(text0))
  text1 = remove_url(text0)
  

  #Punctuation removing
  def remove_punc(text1):
    clean_text = re.sub(r'[?|!|"|:|=|_|{|}|[|]|-|$|%|^|&|]',r' ',str(text1))
    clean_text = re.sub(r'[.|,|)|(|\|/|-|~|`|>|<|*|$|@|;|→]',r' ', clean_text)
    return clean_text
  text2 = remove_url(text1)
  

  def remove_other(text2):
    text = str(text2)
    clean_text = re.sub(r"[^a-zA-Z0-9#+-]", " ", text2.lower())
    return clean_text
  text3 = remove_other(text2)
  

  #Space removing
  def remove_space(text3):
    return ' '.join(str(text3).split())
  text4 = remove_space(text3)
  

  def remove_stopwords(text4):
    text4 = str(text4)
    text4 = " ".join(word for word in text4.split() if word not in gensim.parsing.preprocessing.STOPWORDS and word not in stop_words and word not in set_alpha )
    return text4
  text5 = remove_stopwords(text4)
  

  def tokenize(text5):
    tokens = [lemma(w) for w in text5.split()] #if w.isalpha()
    return tokens
  text6 = tokenize(text5)
  

  def to_string(text6):
    input_string = " ".join(text6)
    return input_string
  text7 = to_string(text6)
  
  def keep_tags(text7):
    text7 = str(text7)
    text7 = " ".join(tag for tag in text7.split() if tag in dict_tags_500)
    return text7
  text8 = keep_tags(text7)
  return text8



In [137]:
tag_question_test = preprocessing(question_test)
print(tag_question_test)

c# c# http proxy http proxy dns


In [138]:
tag_question_test_df = pd.DataFrame([tag_question_test], columns=['question'])
tag_question_test_df

,question
0,c# c# http proxy http proxy dns


In [139]:
tags_500_df = pd.DataFrame([st_tags_500], columns=['question'])
tags_500_df

,question
0,visual-studio-2010 safari asp.net-mvc-3 iis sh...


In [140]:
df_test_input = pd.concat([tag_question_test_df, tags_500_df], ignore_index=True)
df_test_input

,question
0,c# c# http proxy http proxy dns
1,visual-studio-2010 safari asp.net-mvc-3 iis sh...


In [141]:
question_test =  df_test_input['question']
question_test

0                      c# c# http proxy http proxy dns
1    visual-studio-2010 safari asp.net-mvc-3 iis sh...
Name: question, dtype: object

In [142]:
type(question_test)

pandas.core.series.Series

In [143]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [144]:
def tokenize(document):
  tokens = [lemma(w) for w in document.split()] #if w.isalpha()]
  return tokens

In [145]:
stop_words = set(stopwords.words('english'))

In [146]:
vectorizer = TfidfVectorizer(tokenizer = tokenize, stop_words = stop_words, max_features=500)
X_tfidf = vectorizer.fit_transform(question_test).toarray()

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['doe', 'ha', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [147]:
print(X_tfidf.shape)

(2, 498)


In [148]:
feature_names  = vectorizer.get_feature_names()
print(feature_names)

['.htaccess', '.net', '.net-3.5', '.net-core', 'actionscript-3', 'activerecord', 'ajax', 'algorithm', 'amazon-ec2', 'amazon-s3', 'amazon-web-services', 'android', 'android-activity', 'android-emulator', 'android-fragments', 'android-gradle-plugin', 'android-intent', 'android-layout', 'android-studio', 'angular', 'angular-material', 'angular5', 'angularjs', 'animation', 'ansible', 'ant', 'apache', 'apache-kafka', 'apache-spark', 'apache-spark-sql', 'api', 'architecture', 'array', 'arraylist', 'asp.net', 'asp.net-core', 'asp.net-core-2.0', 'asp.net-core-mvc', 'asp.net-mvc', 'asp.net-mvc-2', 'asp.net-mvc-3', 'asp.net-mvc-4', 'asp.net-web-api', 'assembly', 'async-await', 'asynchronous', 'audio', 'authentication', 'autocomplete', 'awk', 'aws-lambda', 'azure', 'bash', 'batch-file', 'binary', 'binding', 'boost', 'bootstrap-4', 'browser', 'build', 'button', 'c', 'c#', 'c++', 'c++11', 'c++14', 'c++17', 'caching', 'cakephp', 'canvas', 'casting', 'chart', 'checkbox', 'clang', 'class', 'clojure', 

In [149]:
len(feature_names)

498

In [151]:
X_test = pd.DataFrame(X_tfidf)
X_test = X_test.iloc[0:1,:]
X_test.columns = feature_names
X_test

,.htaccess,.net,.net-3.5,.net-core,actionscript-3,activerecord,ajax,algorithm,amazon-ec2,amazon-s3,amazon-web-services,android,android-activity,android-emulator,android-fragments,android-gradle-plugin,android-intent,android-layout,android-studio,angular,angular-material,angular5,angularjs,animation,ansible,ant,apache,apache-kafka,apache-spark,apache-spark-sql,api,architecture,array,arraylist,asp.net,asp.net-core,asp.net-core-2.0,asp.net-core-mvc,asp.net-mvc,asp.net-mvc-2,...,vba,vector,version-control,video,vim,visual-c++,visual-studio,visual-studio-2008,visual-studio-2010,visual-studio-2012,visual-studio-2015,visual-studio-2017,visual-studio-code,vue.js,vuejs2,wcf,web,web-applications,web-services,webpack,websocket,webview,winapi,window,windows-7,winforms,wix,woocommerce,wordpress,wpf,x86,xamarin,xamarin.forms,xaml,xcode,xml,xpath,xsd,xslt,zend-framework
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [152]:
X_test.loc[0,'dns']

0.2773500981126146

In [153]:
y_pred = model_lr.predict(X_test)

ValueError: ignored

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer(classes=sorted(dict_tags_500))

In [ ]:
pd.concat([pd.Series(mlb.inverse_transform(y_pred), name='y_pred'),
           pd.Series(mlb.inverse_transform(y_test.to_numpy()), name='y_true')], axis=1)

In [ ]:
A={'.htaccess': 43,'.net': 1201,'.net-3.5': 41}
A          

{'.htaccess': 43, '.net': 1201, '.net-3.5': 41}

In [ ]:
A.keys()

dict_keys(['.htaccess', '.net', '.net-3.5'])

In [ ]:
type(A.keys())

dict_keys